In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

Considering the possible effects connected to LC50. We need to filter on **Mortality Group**

## Preprocessing on base_db

In [2]:
base_db = pd.read_csv("use_db_sabato14.csv")
base_db.head()

,test_cas,exposure_type,obs_duration_mean,conc1_type,conc1_mean,class,tax_order,family,genus,species,atom_number,alone_atom_number,tripleBond,doubleBond,bonds_number,ring_numer,Mol,MorganDensity,LogP
0,10108-64-2,S,48.0,T,46.900,Actinopterygii,Cypriniformes,Cyprinidae,Carassius,auratus,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945
1,10108-64-2,S,24.0,T,200.000,Actinopterygii,Cyprinodontiformes,Cyprinodontidae,Fundulus,heteroclitus,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945
2,10108-64-2,F,96.0,T,0.001,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,mykiss,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945
3,10108-64-2,S,96.0,T,15.900,Actinopterygii,Cyprinodontiformes,Cyprinodontidae,Cyprinodon,variegatus,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945
4,10108-64-2,F,694.0,T,0.030,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,mykiss,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945


In [3]:
base_db = base_db[base_db.obs_duration_mean.isin([24, 48, 72, 96])]

In [4]:
to_drop = base_db[base_db.bonds_number.isnull()].index

In [5]:
base_db = base_db.drop(to_drop,axis=0)

In [6]:
base_db.index.size

45775

In [7]:
base_db.columns

Index(['test_cas', 'exposure_type', 'obs_duration_mean', 'conc1_type',
       'conc1_mean', 'class', 'tax_order', 'family', 'genus', 'species',
       'atom_number', 'alone_atom_number', 'tripleBond', 'doubleBond',
       'bonds_number', 'ring_numer', 'Mol', 'MorganDensity', 'LogP'],
      dtype='object')

In [8]:
use_db = base_db.copy()
use_db = use_db[['test_cas', 'exposure_type', 'obs_duration_mean', 'conc1_type',
       'conc1_mean', 'class', 'tax_order', 'family', 'genus', 'species','atom_number', 'alone_atom_number', 'tripleBond',
       'doubleBond', 'bonds_number', 'ring_numer',
       'Mol', 'MorganDensity', 'LogP']]

In [9]:
use_db.index.size

45775

# Repeated experiments

In [10]:
use_db = use_db.groupby(by=["test_cas", "species", 'class', 'tax_order', 'family', 'genus', "conc1_type", "exposure_type", "obs_duration_mean",]).agg("median").reset_index()
use_db.index.size

23355

### Simple classification with Smile Features
Considering a threshold of 1mg/L of mortality

In [11]:
X = use_db.copy()
X = X[[
 'exposure_type',
 'obs_duration_mean',
 'conc1_type',
 'species',
    'class', 'tax_order', 'family', 'genus',
 'atom_number', 
    'alone_atom_number', 
    'tripleBond',
       'doubleBond',
 'bonds_number',
  'ring_numer',
  'Mol', 
  'MorganDensity', 'LogP']]

In [12]:
X['exposure_type'].unique()

array(['F', 'AQUA', 'S', 'R', 'P', 'IP', 'E', 'SP', 'FD', 'O'],
      dtype=object)

In [13]:
X['conc1_type'].unique()

array(['F', 'A', 'T', 'D', 'L', 'U'], dtype=object)

In [14]:
len(X['species'].unique())

475

For this  vaiables we decided to transform than by an OrdinalEncoder, in order to have all numnerical variables

In [15]:
enc = OrdinalEncoder(dtype=int)
enc.fit(X[["exposure_type", "conc1_type","species", 'class', 'tax_order', 'family', 'genus']])
X[["exposure_type", "conc1_type","species",  'class', 'tax_order', 'family', 'genus']] = enc.transform(X[["exposure_type", "conc1_type","species",  'class', 'tax_order', 'family', 'genus']]) + 1

In [16]:
X.head()

,exposure_type,obs_duration_mean,conc1_type,species,class,tax_order,family,genus,atom_number,alone_atom_number,tripleBond,doubleBond,bonds_number,ring_numer,Mol,MorganDensity,LogP
0,3,48.0,3,379,1,10,36,85,0.317908,2,0,1,0.488106,1.0,0.082505,1.3,2.2482
1,3,96.0,3,379,1,10,36,85,0.317908,2,0,1,0.488106,1.0,0.082505,1.3,2.2482
2,1,48.0,3,218,1,10,36,141,0.317908,2,0,1,0.488106,1.0,0.072092,1.3,1.1770
3,9,24.0,1,237,1,6,2,192,0.317908,2,0,1,0.488106,1.0,0.072092,1.3,1.1770
4,9,48.0,1,237,1,6,2,192,0.317908,2,0,1,0.488106,1.0,0.072092,1.3,1.1770


We also devide the label, in order to have a binary classification

In [17]:
def scores_cat(x):
    if (x < 10**-1):
        return 5
    # elif ((x>=10**-2) and (x<10**-1)):
    #     return 6
    elif ((x>=10**-1) and (x<10**0)):
        return 4
    elif ((x>=10**0) and (x<10**1)):
        return 3
    elif ((x>=10**1) and (x<10**2)):
        return 2
    # elif ((x>=10**2) and (x<10**3)):
    #     return 2
    else:
        return 1

In [18]:
y = use_db["conc1_mean"].copy().values
y = pd.Series(y).apply(lambda x: scores_cat(x)).values

In [19]:
len(X)

23355

In [20]:
print(len(y[y==1]), len(y[y==2]), len(y[y==3]), len(y[y==4]), len(y[y==5]))

3629 5464 5823 4329 4110


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## KNN

We first implement a classification by a K Neighbors Classifier

In [57]:
neigh = KNeighborsClassifier()
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)

In [58]:
accuracy_score(y_test, y_pred)

0.3951948051948052

0.84 is the accuracy on tis first model

## Random Forest

In [24]:
forest = RandomForestClassifier(n_estimators = 100)
forest.fit(X_train, y_train.ravel())
y_pred = forest.predict(X_test)

In [25]:
accuracy_score(y_test, y_pred)

0.6876623376623376

This is the second model we have implemented, with an accurancy of the 0.91. which for the moment is not bad

## Gradient Boosting

In [26]:
grboost = GradientBoostingClassifier()
grboost.fit(X_train, y_train.ravel())
y_pred = grboost.predict(X_test)

In [27]:
accuracy_score(y_test, y_pred)

0.575974025974026

In [28]:
# grid = GridSearchCV(forest, param_grid={"n_estimators": list(range(1, 500))}, scoring="accuracy", cv=5)
# grid.fit(X_train, y_train.ravel())

## FROM HERE WE USE THE DISTANCE MATRIX

In [22]:
categorical = [
'ring_numer',
"exposure_type", 
"conc1_type","species",
'tripleBond',
'obs_duration_mean',
'doubleBond',
'alone_atom_number',
'class', 'tax_order', 'family', 'genus'
 ]

In [23]:
non_categorical =[
 'atom_number',
 'bonds_number',
 'Mol',
 'MorganDensity',
 'LogP']

In [24]:
X_try  =  X#.head(30000)
y_try = y#[:30000]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_try, y_try, test_size=0.33, random_state=42)
X_try = X_train.append(X_test)

In [26]:
import numpy as np
from scipy.spatial.distance import hamming
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

def new_distance_matrix(X, len_X_train, cat_features = [], num_features = [], alpha = 1):
    ''' inputs: matrix X [num_samples, num_features], the list of the categorical features, the list of the numerical features, weight alpha
        output: distance matrix
    '''

    # Training
    X_cat = X[cat_features]
    X_num = X[num_features]
    dist_matr = alpha * squareform(pdist(X_cat, metric = "hamming"))
    dist_matr += squareform(pdist(X_num, metric = "euclidean"))

    dist_matr_train = dist_matr[:len_X_train,:len_X_train]
    dist_matr_test = dist_matr[len_X_train:,:len_X_train]

    return dist_matr_train, dist_matr_test


In [27]:
# len_X_train = len(X_train)
# X_train_new, X_test_new = new_distance_matrix(X_try, len_X_train, categorical,non_categorical, alpha = 5)

In [28]:
X_train_new.shape

NameError: name 'X_train_new' is not defined

In [29]:
X_test_new.shape

NameError: name 'X_test_new' is not defined

In [30]:
neigh = KNeighborsClassifier(metric = 'precomputed')
neigh.fit(X_train_new, y_train.ravel())
y_pred = neigh.predict(X_test_new)

NameError: name 'X_train_new' is not defined

In [31]:
accuracy_score(y_test, y_pred)

NameError: name 'y_pred' is not defined

## Selecting best features for the problem

In [27]:
import numpy as np
from scipy.spatial.distance import hamming
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

def new_distance_matrix_selective(X, len_X_train, cat_features = [], num_features = [], alpha = 1):
    ''' inputs: matrix X [num_samples, num_features], the list of the categorical features, the list of the numerical features, weight alpha
        output: distance matrix
    '''

    # Training
    X_cat = X[cat_features]
    X_num = X[num_features]
    
    if (len(cat_features)!=0 and len(num_features)):
        dist_matr = alpha * squareform(pdist(X_cat, metric = "hamming"))
        dist_matr += squareform(pdist(X_num, metric = "euclidean"))
    elif(len(cat_features)!=0):
        dist_matr = alpha * squareform(pdist(X_cat, metric = "hamming"))
    else:
        dist_matr = squareform(pdist(X_num, metric = "euclidean"))

    dist_matr_train = dist_matr[:len_X_train,:len_X_train]
    dist_matr_test = dist_matr[len_X_train:,:len_X_train]

    return dist_matr_train, dist_matr_test


In [28]:
import itertools
np.random.seed(13)

best_acc = 0
best_comb = []
best_cat = []
best_non_cat = []

poss_features = np.array(categorical + non_categorical)
for k in range(1, 18):
    print("Starting k =",k)
    poss_comb = list(itertools.combinations(range(0,17),k))
    for c in poss_comb:
        cat = []
        non_cat = []
    for i in list(c):
        if i in range(0, 12):
            cat.append(poss_features[i])
        else:
            non_cat.append(poss_features[i])

    len_X_train = len(X_train)
    X_train_new, X_test_new = new_distance_matrix_selective(X_try, len_X_train, cat,non_cat, alpha = 1)
    
    neigh = KNeighborsClassifier(metric = 'precomputed')
    neigh.fit(X_train_new, y_train.ravel())
    y_pred = neigh.predict(X_test_new)
    
    acc = accuracy_score(y_test, y_pred)

    if acc>best_acc:
        best_acc = acc
        best_cat = cat
        best_non_cat = non_cat
        print("Best combination found! Acc: {}, features: cat: {}, non_cat:{}".format(best_acc, best_cat, best_non_cat))
    
    del X_train_new, X_test_new

Starting k = 1
Best combination found! Acc: 0.571613907628438, features: cat: [], non_cat:['LogP']
Starting k = 2
Best combination found! Acc: 0.577322262584328, features: cat: [], non_cat:['MorganDensity', 'LogP']
Starting k = 3
Best combination found! Acc: 0.5941878567721848, features: cat: [], non_cat:['Mol', 'MorganDensity', 'LogP']
Starting k = 4
Best combination found! Acc: 0.5944473274519979, features: cat: [], non_cat:['bonds_number', 'Mol', 'MorganDensity', 'LogP']
Starting k = 5
Best combination found! Acc: 0.5948365334717177, features: cat: [], non_cat:['atom_number', 'bonds_number', 'Mol', 'MorganDensity', 'LogP']
Starting k = 6
Starting k = 7
Starting k = 8
Starting k = 9
Starting k = 10
Best combination found! Acc: 0.6050856253243383, features: cat: ['alone_atom_number', 'class', 'tax_order', 'family', 'genus'], non_cat:['atom_number', 'bonds_number', 'Mol', 'MorganDensity', 'LogP']
Starting k = 11
Best combination found! Acc: 0.6201349247535028, features: cat: ['doubleBo

## CV

In [29]:
from sklearn.model_selection import KFold

def cv_new_distance(X, y, cat_features = [], num_features = [], alphas = [], ks = [], leafs=[]):

    best_accuracy = 0
    best_alpha = 0
    best_k = 0
    best_leaf = 0
    
    X_cat = X[cat_features]
    X_num = X[num_features]
    dist_matr_num = squareform(pdist(X_num, metric = "euclidean"))


    for alpha in alphas:
        for k in ks:
            for leaf in leafs:
                
                dist_matr = alpha * squareform(pdist(X_cat, metric = "hamming"))
                dist_matr += dist_matr_num
                dist_matr = pd.DataFrame(dist_matr)

                kf = KFold(n_splits=5, shuffle=True)
                accs = []
                for train_index, test_index in kf.split(dist_matr):
                    
                    X_train = dist_matr.iloc[train_index, train_index]
                    X_test = dist_matr.iloc[test_index, train_index]
                    y_train = y[train_index]
                    y_test = y[test_index]

                    neigh = KNeighborsClassifier(metric = 'precomputed', n_neighbors=k, n_jobs=-2, leaf_size=leaf)
                    neigh.fit(X_train, y_train.ravel())
                    y_pred = neigh.predict(X_test)

                    accs.append(accuracy_score(y_test, y_pred))

                avg_acc = np.mean(accs)
                if (avg_acc > best_accuracy):
                    print("New best params found! alpha:{}, k:{}, leaf:{}, acc:{}".format(alpha, k, leaf, avg_acc))
                    best_alpha = alpha
                    best_k = k
                    best_accuracy = avg_acc
                    best_leaf = leaf

    return best_accuracy, best_alpha, best_k, best_leaf


In [30]:
alphas = np.logspace(-3, 1, 20)
ks = range(1, 10)
leafs = range(10, 51, 10)
best_acc, best_alpha, best_k, best_leaf = cv_new_distance(X_train, y_train, categorical, non_categorical, alphas, ks, leafs)

New best params found! alpha:0.001, k:1, leaf:10, acc:0.6492441842978651
New best params found! alpha:0.001623776739188721, k:1, leaf:20, acc:0.649499795498763
New best params found! alpha:0.001623776739188721, k:1, leaf:40, acc:0.6497564910984162
New best params found! alpha:0.0026366508987303583, k:1, leaf:30, acc:0.6506517134216443
New best params found! alpha:0.004281332398719396, k:1, leaf:10, acc:0.6518032835185098


# First test (No feature selection -> 13 features)

In [31]:
best_alpha = 0.004281332398719396

In [32]:
best_k = 1

In [33]:
best_leaf = 10

In [34]:
len_X_train = len(X_train)
X_train_new, X_test_new = new_distance_matrix(X_try, len_X_train, categorical,non_categorical, alpha = best_alpha)
neigh = KNeighborsClassifier(metric = 'precomputed', n_neighbors=best_k)
neigh.fit(X_train_new, y_train.ravel())
y_pred = neigh.predict(X_test_new)

In [35]:
accuracy_score(y_test, y_pred)

0.6636363636363637

In [36]:
from math import sqrt
from sklearn.metrics import mean_squared_error
sqrt(mean_squared_error(y_test, y_pred))

0.777775716344413